## Examining the Effects of Additive Noise on Exponential Smoothing Forecast Accuracy

***

In [1]:
# general modules
import pandas as pd
import numpy as np
import sktime

# import exponential smoothing forecasting model
from sktime.forecasting.exp_smoothing import ExponentialSmoothing

# functions for transformation+forecasting pipeline
from sktime.forecasting.compose import TransformedTargetForecaster

# time series transformations
from sktime.transformations.series.detrend import ConditionalDeseasonalizer

##### the `helper_functions.py` file contains many custom functions we wrote to aid in our analysis
##### `full_coding_analysis` combines all of the following - train-test split data,
##### data protection, train models, compare accuracies, return accuracy results
from helper_functions import *

# suppress warnings from exponential smoothing model not converging
import warnings
warnings.filterwarnings('ignore')

# nice time series plots
from sktime.utils.plotting import plot_series

In [2]:
# import weekly finance time series
Y = np.genfromtxt("../../Data/Train/Clean/weekly_finance_clean.csv", delimiter = ',', skip_header = 1)
Y = pd.DataFrame(Y)

***
***

In [3]:
# time_series_data = Y
# forecast_horizon=20
# coding_type="Top"
# coding_percentage=0.10
# forecasting_model = lgb.LGBMRegressor()
# # define forecasting model
# # perform additive deseasonalization conditional on autocorrelation test for seasonality

# forecasting_model = TransformedTargetForecaster(
#     [
#         ("forecast", ExponentialSmoothing(use_boxcox=False)),
#     ]
# )
# window_length=10

In [4]:
# # create train-test split - we assume the 'test' values are unobserved
#     # transpose the input data because `temporal_train_test_split` splits on rows
# Train, Test = temporal_train_test_split(time_series_data.T, test_size=forecast_horizon)
#     # transpose back
# Train = Train.T
# Test = Test.T

In [5]:
# if coding_type is not None:
#     Train_protected = Train.apply(coding_protection, axis=1, args=(coding_type, coding_percentage))
# elif num_stdev is not None:
#     Train_protected = additive_noise_protection(Train, num_stdev=num_stdev)
# elif epsilon is not None:
#     Train_protected = DP_protection(Train, epsilon=epsilon)

In [6]:
# Train, mins, means = pre_process(Train)
# Train_protected, mins_protected, means_protected = pre_process(Train_protected)

In [7]:
# if type(forecasting_model) == lgb.sklearn.LGBMRegressor:
#     #Train, Test = pre_process(Train, Test)
#     #Train_protected, _ = pre_process(Train_protected, Test)
#     # construct detrender
#     detrender = Detrender()
#     detrended_series = [detrender.fit_transform(series) for _ , series in Train_protected.iterrows()]
#     Train_protected_orig = Train_protected.copy()
#     Train_protected = pd.concat(detrended_series, axis=1).T
#     detrended_series = [detrender.fit_transform(series) for _ , series in Train.iterrows()]
#     Train_orig = Train.copy()
#     Train = pd.concat(detrended_series, axis=1).T
    
# if type(forecasting_model) == sktime.forecasting.compose._pipeline.TransformedTargetForecaster:
#     # create nested dataframes to use with sktime functions
#     Train = from_2d_array_to_nested(Train)
#     Train_protected = from_2d_array_to_nested(Train_protected)

In [8]:
# # forecasts from model trained on original data
# fcasts = train_and_forecast(forecasting_model=forecasting_model, horizon_length=forecast_horizon, training_data=Train, window_length=window_length)

#     # forecasts from model trained on protected data
# fcasts_protected = train_and_forecast(forecasting_model=forecasting_model, horizon_length=forecast_horizon, training_data=Train_protected, window_length=window_length)

In [9]:
# if type(forecasting_model) == lgb.sklearn.LGBMRegressor:
#     fcasts = reverse_transformation(fcasts, Train_orig, "Add Trend")
#     fcasts_protected = reverse_transformation(fcasts_protected, Train_protected_orig, "Add Trend")

In [10]:
# fcasts = post_process(fcasts, mins, means)
# fcasts_protected = post_process(fcasts_protected, mins_protected, means_protected)

***
***

In [11]:
# detrender = Detrender()
# detrended_series = [detrender.fit_transform(series) for _ , series in Y.iterrows()]
# detrended_series = [i+np.abs(np.min(i))+1.0 for i in detrended_series]
# Y = pd.concat(detrended_series, axis=1).T

***

## SES

In [12]:
# define forecasting model
# perform additive deseasonalization conditional on autocorrelation test for seasonality

forecaster = TransformedTargetForecaster(
    [
        ("forecast", ExponentialSmoothing(use_boxcox=False)),
    ]
)

In [13]:
results_dict_ses = {}
fcasts_ses = {}
fcasts_protected_ses = {}
tests = {}
num_stdevs = [1, 2]
horizons = [1, 20]

In [14]:
for n in num_stdevs:
    for h in horizons:
        idx = "h="+str(h)+", "+str(n)+" stan. devs"
        results_dict_ses[idx], tests[idx], fcasts_ses[idx], fcasts_protected_ses[idx] = full_coding_analysis(time_series_data=Y, 
                                                                                                             forecasting_model=forecaster, 
                                                                                                             forecast_horizon=h,
                                                                                                             num_stdev=n)

***
***

In [15]:
results_dict_ses

{'h=1, 1 stan. devs': {'Mean Accuracies': array([55.3207, 55.3207]),
  'Protected Mean Accuracies:': array([109.6771, 109.6771]),
  '% Change Mean accuracy:': array([-0.9826, -0.9826]),
  '% Change Median accuracy:': array([-1.6811, -1.6811]),
  '% Forecasted Points adjusted downward:': 0.6402,
  '% Forecasted Points adjusted upward:': 0.3598,
  '% Series with improved accuracy:': array([0.2439, 0.2439]),
  '% Series with reduced accuracy:': array([0.7561, 0.7561]),
  'Original Mean Absolute Error Upward Adjusted:': 65.4895,
  'Original Mean Absolute Error Downward Adjusted:': 49.6068,
  'Protected Mean Absolute Error Upward Adjusted:': 115.0888,
  'Protected Mean Absolute Error Downward Adjusted:': 106.6362},
 'h=20, 1 stan. devs': {'Mean Accuracies': array([112.5495, 135.001 ]),
  'Protected Mean Accuracies:': array([149.1244, 167.7697]),
  '% Change Mean accuracy:': array([-0.325 , -0.2427]),
  '% Change Median accuracy:': array([-0.3574, -0.2399]),
  '% Forecasted Points adjusted d

***
***

## DES

In [16]:
# define forecasting model
# perform additive deseasonalization conditional on autocorrelation test for seasonality

forecaster = TransformedTargetForecaster(
    [
        ("forecast", ExponentialSmoothing(trend="additive", use_boxcox=False)),
    ]
)

In [17]:
results_dict_des = {}
fcasts_des = {}
fcasts_protected_des = {}
tests = {}
num_stdevs = [1, 2]
horizons = [1, 20]

In [18]:
for n in num_stdevs:
    for h in horizons:
        idx = "h="+str(h)+", "+str(n)+" stan. devs"
        results_dict_des[idx], tests[idx], fcasts_des[idx], fcasts_protected_des[idx] = full_coding_analysis(time_series_data=Y, 
                                                                                                             forecasting_model=forecaster, 
                                                                                                             forecast_horizon=h,
                                                                                                             num_stdev=n)

In [19]:
results_dict_des

{'h=1, 1 stan. devs': {'Mean Accuracies': array([54.4492, 54.4492]),
  'Protected Mean Accuracies:': array([131.9051, 131.9051]),
  '% Change Mean accuracy:': array([-1.4225, -1.4225]),
  '% Change Median accuracy:': array([-1.702, -1.702]),
  '% Forecasted Points adjusted downward:': 0.5732,
  '% Forecasted Points adjusted upward:': 0.4268,
  '% Series with improved accuracy:': array([0.2317, 0.2317]),
  '% Series with reduced accuracy:': array([0.7683, 0.7683]),
  'Original Mean Absolute Error Upward Adjusted:': 63.2068,
  'Original Mean Absolute Error Downward Adjusted:': 47.9275,
  'Protected Mean Absolute Error Upward Adjusted:': 127.4885,
  'Protected Mean Absolute Error Downward Adjusted:': 135.1941},
 'h=20, 1 stan. devs': {'Mean Accuracies': array([109.1455, 132.0211]),
  'Protected Mean Accuracies:': array([165.5499, 188.5545]),
  '% Change Mean accuracy:': array([-0.5168, -0.4282]),
  '% Change Median accuracy:': array([-0.2728, -0.2722]),
  '% Forecasted Points adjusted dow

***
***

## TES

In [20]:
# define forecasting model
# perform additive deseasonalization conditional on autocorrelation test for seasonality

forecaster = TransformedTargetForecaster(
    [
        ("forecast", ExponentialSmoothing(trend="additive",
                                          seasonal="additive",
                                          sp=52,
                                          damped_trend=False, 
                                          use_boxcox=False)),
    ]
)

In [21]:
results_dict_tes = {}
fcasts_tes = {}
fcasts_protected_tes = {}
tests = {}
num_stdevs = [1, 2]
horizons = [1, 20]

In [22]:
for n in num_stdevs:
    for h in horizons:
        idx = "h="+str(h)+", "+str(n)+" stan. devs"
        results_dict_tes[idx], tests[idx], fcasts_tes[idx], fcasts_protected_tes[idx] = full_coding_analysis(time_series_data=Y, 
                                                                                                             forecasting_model=forecaster, 
                                                                                                             forecast_horizon=h,
                                                                                                             num_stdev=n)

In [23]:
results_dict_tes

{'h=1, 1 stan. devs': {'Mean Accuracies': array([66.4076, 66.4076]),
  'Protected Mean Accuracies:': array([215.2398, 215.2398]),
  '% Change Mean accuracy:': array([-2.2412, -2.2412]),
  '% Change Median accuracy:': array([-2.0045, -2.0045]),
  '% Forecasted Points adjusted downward:': 0.5915,
  '% Forecasted Points adjusted upward:': 0.4085,
  '% Series with improved accuracy:': array([0.1463, 0.1463]),
  '% Series with reduced accuracy:': array([0.8537, 0.8537]),
  'Original Mean Absolute Error Upward Adjusted:': 87.0271,
  'Original Mean Absolute Error Downward Adjusted:': 52.1653,
  'Protected Mean Absolute Error Upward Adjusted:': 189.4094,
  'Protected Mean Absolute Error Downward Adjusted:': 233.0815},
 'h=20, 1 stan. devs': {'Mean Accuracies': array([116.7834, 138.5197]),
  'Protected Mean Accuracies:': array([246.0935, 300.8327]),
  '% Change Mean accuracy:': array([-1.1073, -1.1718]),
  '% Change Median accuracy:': array([-1.5984, -1.6331]),
  '% Forecasted Points adjusted d